In [ ]:
import pandas as pd
import glob
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import torch

# Prüfen, ob CUDA (GPU) verfügbar ist
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Verwendetes Gerät: {device}")

# 1. Excel-Dateien laden und max. 1000 Zeilen verwenden
pfad = r"your_path_to_excel_files\*.xlsx"  # Pfad zu den Excel-Dateien anpassen
dateien = glob.glob(pfad)
dfs = [pd.read_excel(datei) for datei in dateien]
df = pd.concat(dfs, ignore_index=True)

# 2. Nur relevante Spalten & Labels
df = df[['text', 'label_num']].dropna()
df = df[df['text'].str.strip().astype(bool)]
df = df.rename(columns={'label_num': 'labels'})

# Max. 1000 zufällige Zeilen zum Testen
#df = df.sample(n=10000, random_state=42)

# 3. Aufteilen in Train/Validierung
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df["text"], df["labels"], test_size=0.2, stratify=df["labels"], random_state=42
)

# 4. DistilBERT Tokenizer
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True)
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True)

# 5. Datensätze
train_dataset = Dataset.from_dict({**train_encodings, "labels": list(train_labels)})
val_dataset = Dataset.from_dict({**val_encodings, "labels": list(val_labels)})

# 6. Modell
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2).to(device)

# 7. Trainingsargumente
training_args = TrainingArguments(
    output_dir=r"your_output_directory",  # Pfad zum Speichern der Ergebnisse anpassen
    num_train_epochs=1,  # schnellere Tests
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    evaluation_strategy="epoch",
    logging_dir="./bert_logs",
    logging_steps=10,
    save_total_limit=1,
    load_best_model_at_end=False,
)

# 8. Metriken
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# 9. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

# 10. Training starten
trainer.train()

# 11. Evaluation
eval_result = trainer.evaluate()
print(eval_result)

from transformers import DistilBertForSequenceClassification, DistilBertTokenizerFast

# Speicherort
save_dir = r"your_save_directory"  # Pfad zum Speichern des Modells und Tokenizers anpassen

# Modell und Tokenizer speichern
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

print(f"Modell und Tokenizer wurden gespeichert in: {save_dir}")



Verwendetes Gerät: cuda


C:\Users\HP\AppData\Roaming\Python\Python310\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/1660 [00:00<?, ?it/s]

{'loss': 0.5825, 'grad_norm': 2.429337739944458, 'learning_rate': 4.9698795180722894e-05, 'epoch': 0.01}
{'loss': 0.2689, 'grad_norm': 2.8033030033111572, 'learning_rate': 4.9397590361445786e-05, 'epoch': 0.01}
{'loss': 0.0953, 'grad_norm': 4.809431552886963, 'learning_rate': 4.909638554216868e-05, 'epoch': 0.02}
{'loss': 0.1426, 'grad_norm': 13.496682167053223, 'learning_rate': 4.879518072289157e-05, 'epoch': 0.02}
{'loss': 0.2016, 'grad_norm': 2.674021005630493, 'learning_rate': 4.8493975903614455e-05, 'epoch': 0.03}
{'loss': 0.1802, 'grad_norm': 0.9700006246566772, 'learning_rate': 4.8192771084337354e-05, 'epoch': 0.04}
{'loss': 0.1575, 'grad_norm': 3.352940797805786, 'learning_rate': 4.7891566265060246e-05, 'epoch': 0.04}
{'loss': 0.1025, 'grad_norm': 5.5608391761779785, 'learning_rate': 4.759036144578313e-05, 'epoch': 0.05}
{'loss': 0.141, 'grad_norm': 3.1970674991607666, 'learning_rate': 4.728915662650602e-05, 'epoch': 0.05}
{'loss': 0.1303, 'grad_norm': 7.484382629394531, 'learn

  0%|          | 0/415 [00:00<?, ?it/s]

{'eval_loss': 0.01474866271018982, 'eval_accuracy': 0.9953306220816388, 'eval_f1': 0.9953516269305743, 'eval_precision': 0.9972956730769231, 'eval_recall': 0.993415145166118, 'eval_runtime': 119.7513, 'eval_samples_per_second': 55.44, 'eval_steps_per_second': 3.466, 'epoch': 1.0}
{'train_runtime': 4209.2081, 'train_samples_per_second': 6.308, 'train_steps_per_second': 0.394, 'train_loss': 0.05361222588760128, 'epoch': 1.0}


  0%|          | 0/415 [00:00<?, ?it/s]

{'eval_loss': 0.01474866271018982, 'eval_accuracy': 0.9953306220816388, 'eval_f1': 0.9953516269305743, 'eval_precision': 0.9972956730769231, 'eval_recall': 0.993415145166118, 'eval_runtime': 119.7504, 'eval_samples_per_second': 55.44, 'eval_steps_per_second': 3.466, 'epoch': 1.0}
Modell und Tokenizer wurden gespeichert in: C:\Users\HP\Desktop\AWA_BERT\bert_results
